## For illusoins online hotel

In [1]:
import pandas as pd
import os

file = "D:/Rokon/hotels_content_to_create_json_file/IllusionsOnline/illusions-online-aWxzLTMwMzM1.csv"



['hotelCode;hotelName;country;latitude;longitude;address;categoryCode;giataCode']


In [2]:
df = pd.read_csv(file, nrows = 1)
headers = df.columns.tolist()
print(headers)

['hotelCode;hotelName;country;latitude;longitude;address;categoryCode;giataCode']


### Add Column anly

In [2]:
from sqlalchemy import create_engine, MetaData, inspect
from sqlalchemy.sql import text
import pandas as pd

# File and database setup
file = "D:/Rokon/hotels_content_to_create_json_file/IllusionsOnline/illusions-online-aWxzLTMwMzM1.csv"
df = pd.read_csv(file, delimiter=';', nrows=1) 
headers = df.columns.tolist()

# Split and sanitize headers if they appear as a single string
if len(headers) == 1 and ';' in headers[0]:
    headers = headers[0].split(';')  # Split single string into list
headers = [col.strip().replace(" ", "_").replace("-", "_") for col in headers]  

DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)
metadata = MetaData()
metadata.reflect(bind=engine)

# Inspect existing table
table_name = "illusions_online"
inspector = inspect(engine)
if table_name in inspector.get_table_names():
    existing_columns = [col["name"] for col in inspector.get_columns(table_name)]

    # Dynamically add missing columns
    with engine.connect() as conn:
        for column_name in headers:
            if column_name not in existing_columns:
                column_type = "FLOAT" if column_name in ['latitude', 'longitude', 'categoryCode', 'giataCode'] else "VARCHAR(255)"
                try:
                    # Prepare and execute ALTER TABLE statement
                    alter_statement = text(f"ALTER TABLE `{table_name}` ADD COLUMN `{column_name}` {column_type}")
                    conn.execute(alter_statement)
                    print(f"Added column: {column_name} ({column_type})")
                except Exception as e:
                    print(f"Failed to add column {column_name}: {e}")
else:
    print(f"Table '{table_name}' does not exist.")

print("Columns update completed.")


Added column: hotelCode (VARCHAR(255))
Added column: hotelName (VARCHAR(255))
Added column: country (VARCHAR(255))
Added column: latitude (FLOAT)
Added column: longitude (FLOAT)
Added column: address (VARCHAR(255))
Added column: categoryCode (FLOAT)
Added column: giataCode (FLOAT)
Columns update completed.


### Upload CSV file to database 

In [3]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table
import numpy as np

DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL, pool_size=10, max_overflow=20)  # Connection pool configuration

chunk_size = 1000
file = "D:/Rokon/hotels_content_to_create_json_file/IllusionsOnline/illusions-online-aWxzLTMwMzM1.csv"

# Load the table metadata
metadata = MetaData()

# Reflect the table (without bind)
illutions_online = Table('illusions_online', metadata, autoload_with=engine)

# Read CSV in chunks, skipping bad lines
for chunk in pd.read_csv(file, chunksize=chunk_size, on_bad_lines='skip', delimiter=';'):
    # Handle missing values or data type conversions using `map` instead of `applymap`
    chunk = chunk.apply(lambda x: None if isinstance(x, float) and np.isnan(x) else x)
    
    # Check the column names in the chunk for debugging
    print("Columns in the CSV:", chunk.columns)

    # Insert the chunk into the table
    try:
        chunk.to_sql('illusions_online', con=engine, if_exists='append', index=False)
        print(f"Inserted a chunk of {len(chunk)} rows into the 'illutions_online' table.")
    except Exception as e:
        print(f"Error inserting data: {e}")

print("Data insertion complete.")


Columns in the CSV: Index(['hotelCode', 'hotelName', 'country', 'latitude', 'longitude', 'address',
       'categoryCode', 'giataCode'],
      dtype='object')
Inserted a chunk of 1000 rows into the 'illutions_online' table.
Columns in the CSV: Index(['hotelCode', 'hotelName', 'country', 'latitude', 'longitude', 'address',
       'categoryCode', 'giataCode'],
      dtype='object')
Inserted a chunk of 1000 rows into the 'illutions_online' table.
Columns in the CSV: Index(['hotelCode', 'hotelName', 'country', 'latitude', 'longitude', 'address',
       'categoryCode', 'giataCode'],
      dtype='object')
Inserted a chunk of 1000 rows into the 'illutions_online' table.
Columns in the CSV: Index(['hotelCode', 'hotelName', 'country', 'latitude', 'longitude', 'address',
       'categoryCode', 'giataCode'],
      dtype='object')
Inserted a chunk of 1000 rows into the 'illutions_online' table.
Columns in the CSV: Index(['hotelCode', 'hotelName', 'country', 'latitude', 'longitude', 'address',
    

### Illusions hotel table to local iit

In [4]:
from sqlalchemy import create_engine, Table, MetaData, insert
from sqlalchemy.orm import sessionmaker
import pandas as pd

DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()
metadata = MetaData()

metadata.reflect(engine)

illusionsHotel = Table('illusions_online', metadata, autoload=True, autoload_with=engine)
innovativeHotel = Table('innova_hotels_main', metadata, autoload=True, autoload_with=engine)

def transfer_all_data():
    try:
        # Get total rows using raw SQL query
        total_rows = session.execute(illusionsHotel.select()).rowcount
        batch_size = 1000
        total_batches = (total_rows // batch_size) + (1 if total_rows % batch_size > 0 else 0)
        
        for batch in range(total_batches):
            offset = batch * batch_size
            # Use SQLAlchemy Core to construct the query
            query = illusionsHotel.select().limit(batch_size).offset(offset)
            df = pd.read_sql(query, con=engine)
            rows = df.astype(str).to_dict(orient='records')

            # Process rows without nested transactions
            for row in rows:
                keys_to_extract = ['Id', 'hotelCode', 'supplierCode', 'hotelName', 'country', 'latitude', 'longitude', 'address', 'categoryCode', 'giataCode']
                filtered_row_dict = {key: row.get(key, None) for key in keys_to_extract}

                data = {
                    'HotelId': filtered_row_dict.get("hotelCode", None),
                    'GiataCode': filtered_row_dict.get("giataCode", None),
                    'CountryCode': filtered_row_dict.get("country", None),
                    'HotelName': filtered_row_dict.get("hotelName", None),
                    'Latitude': filtered_row_dict.get("latitude", None),
                    'Longitude': filtered_row_dict.get("longitude", None),
                    'AddressLine1': filtered_row_dict.get("address", None),
                    'SupplierCode': filtered_row_dict.get("supplierCode", None)
                }

                stmt = insert(innovativeHotel).values(data)
                session.execute(stmt)
            session.commit()  # Commit after processing the batch
            print(f"Batch {batch + 1} of {total_batches} completed")
        
        print("Data transfer completed")
    except Exception as e:
        print(f"Error occurred: {e}")
        session.rollback()
    finally:
        session.close()

transfer_all_data()


Batch 1 of 24 completed
Batch 2 of 24 completed
Batch 3 of 24 completed
Batch 4 of 24 completed
Batch 5 of 24 completed
Batch 6 of 24 completed
Batch 7 of 24 completed
Batch 8 of 24 completed
Batch 9 of 24 completed
Batch 10 of 24 completed
Batch 11 of 24 completed
Batch 12 of 24 completed
Batch 13 of 24 completed
Batch 14 of 24 completed
Batch 15 of 24 completed
Batch 16 of 24 completed
Batch 17 of 24 completed
Batch 18 of 24 completed
Batch 19 of 24 completed
Batch 20 of 24 completed
Batch 21 of 24 completed
Batch 22 of 24 completed
Batch 23 of 24 completed
Batch 24 of 24 completed
Data transfer completed


## Server IIT table to Vervotech mapping Table.

In [7]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
from datetime import datetime
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Database credentials from .env
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

# Create SQLAlchemy engine
SERVER_DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
server_engine = create_engine(SERVER_DATABASE_URL)

def insert_data_in_chunks(engine, chunk_size, page_size):
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("Starting data insertion process...")

    select_query = text("""
        SELECT * FROM innova_hotels_main
        WHERE SupplierCode =  'illusionshotel'
        LIMIT :limit OFFSET :offset
    """)

    insert_query = text("""
        INSERT INTO vervotech_mapping (
            last_update, VervotechId, GiataCode, ProviderHotelId, ProviderFamily, status,
            ModifiedOn, hotel_city, hotel_name, hotel_country, hotel_longitude,
            hotel_latitude, country_code, content_update_status, created_at
        )
        VALUES (:last_update, :VervotechId, :GiataCode, :ProviderHotelId, :ProviderFamily, :status,
                :ModifiedOn, :hotel_city, :hotel_name, :hotel_country, :hotel_longitude,
                :hotel_latitude, :country_code, :content_update_status, :created_at)
    """)

    try:
        with engine.connect() as connection:
            offset = 0
            chunk_count = 1

            while True:
                # Fetch a small batch of data at a time
                result = connection.execute(select_query, {'limit': page_size, 'offset': offset}).fetchall()

                if not result:
                    print("No more records to transfer.")
                    break

                print(f"Fetched {len(result)} records from the database.")
                offset += page_size

                data_to_insert = []
                for row in result:
                    data_to_insert.append({
                        'last_update': current_time,
                        'VervotechId': None,
                        'GiataCode': row[3],
                        'ProviderHotelId': row[5], 
                        'ProviderFamily': row[4], 
                        'status': "Update",
                        'ModifiedOn': current_time,
                        'hotel_city': row[7],  
                        'hotel_name': row[15],
                        'hotel_country': row[12],  
                        'hotel_longitude': row[17],
                        'hotel_latitude': row[16], 
                        'country_code': row[13], 
                        'content_update_status': "Done",
                        'created_at': current_time  
                    })

                if data_to_insert:
                    try:
                        # Execute the bulk insert
                        connection.execute(insert_query, data_to_insert)
                        print(f"Chunk {chunk_count} inserted successfully.")
                        data_to_insert.clear()
                        chunk_count += 1
                        # Commit the transaction after each chunk
                        connection.commit()
                    except SQLAlchemyError as e:
                        print(f"Error during insert operation for chunk {chunk_count}: {e}")
                        connection.rollback()

    except SQLAlchemyError as e:
        print(f"Error during database operation: {e}")

    print("Data insertion completed.")

# Call the function to process data in chunks
insert_data_in_chunks(server_engine, chunk_size=1000, page_size=1000)


Starting data insertion process...
Fetched 1000 records from the database.
Chunk 1 inserted successfully.
Fetched 1000 records from the database.
Chunk 2 inserted successfully.
Fetched 1000 records from the database.
Chunk 3 inserted successfully.
Fetched 1000 records from the database.
Chunk 4 inserted successfully.
Fetched 1000 records from the database.
Chunk 5 inserted successfully.
Fetched 1000 records from the database.
Chunk 6 inserted successfully.
Fetched 1000 records from the database.
Chunk 7 inserted successfully.
Fetched 1000 records from the database.
Chunk 8 inserted successfully.
Fetched 1000 records from the database.
Chunk 9 inserted successfully.
Fetched 1000 records from the database.
Chunk 10 inserted successfully.
Fetched 1000 records from the database.
Chunk 11 inserted successfully.
Fetched 1000 records from the database.
Chunk 12 inserted successfully.
Fetched 1000 records from the database.
Chunk 13 inserted successfully.
Fetched 1000 records from the database